# This notebook

This is the continuation of IrrToProduction but by focusing onn a single plant at a time.

## Goals

- Predict AC_POWER with good accuracy and without offerfitting
- Forecast AC_POWER

### Predict AC_POWER

In [2]:
# Import

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import  make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error

In [3]:
# Import & Transforms data
def preprocess(d, plant_nb):
    d["generator"] = d["generator"].drop(columns=["SOURCE_KEY", "PLANT_ID", "DC_POWER", "TOTAL_YIELD", "DAILY_YIELD"])
    d['generator']['DATE_TIME'] = pd.to_datetime(d['generator']['DATE_TIME'], format='%d-%m-%Y %H:%M') if plant_nb==1 else pd.to_datetime(d['generator']['DATE_TIME'], format='%Y-%m-%d %H:%M:%S')
    d["sensor"] = d["sensor"].drop(columns=["time", "date"])
    X = d['generator'].merge(d["sensor"], on="DATE_TIME").drop(columns=["DATE_TIME"])
    y = X.pop("AC_POWER")
    return X, y 


p1_X, p1_y = preprocess({"generator": pd.read_csv("dataset/Plant_1_Generation_Data.csv"), 
           "sensor": pd.read_parquet("dataset/parquets/plant_1_updated_sensor")}, 1)

p2_X, p2_y = preprocess({"generator": pd.read_csv("dataset/Plant_2_Generation_Data.csv"), 
           "sensor": pd.read_parquet("dataset/parquets/plant_2_updated_sensor")}, 2)

x_train, x_test, y_train, y_test = train_test_split(p1_X, p1_y, test_size=.5)

In [4]:
# Train
param_dist = {
    'n_estimators': np.arange(50, 200, 5),
    'max_depth': np.arange(2, 10, 1),
    'learning_rate': np.linspace(0.01, 1, 25),
    'subsample': np.arange(0.1, 1.0, 0.25), 
    'max_features': np.linspace(0.1, 1.0, 5),
    'lambda_l2': np.linspace(0.01, 0.5, 5),
}

model_search = RandomizedSearchCV(
    lgb.LGBMRegressor(), 
    param_dist, 
    n_iter=500,
    cv=None, 
    n_jobs=-1,
)

model_search.fit(x_train, y_train)


[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] lambda_l2 is set=0.01, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] lambda_l2 is set=0.01, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000469 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 861
[LightGBM] [Info] Number of data points in the train set: 34389, number of used features: 4
[LightGBM] [Info] Start training from score 306.373374
[LightGBM] [Warn

RandomizedSearchCV(estimator=LGBMRegressor(), n_iter=500, n_jobs=-1,
                   param_distributions={'lambda_l2': array([0.01  , 0.1325, 0.255 , 0.3775, 0.5   ]),
                                        'learning_rate': array([0.01   , 0.05125, 0.0925 , 0.13375, 0.175  , 0.21625, 0.2575 ,
       0.29875, 0.34   , 0.38125, 0.4225 , 0.46375, 0.505  , 0.54625,
       0.5875 , 0.62875, 0.67   , 0.71125, 0.7525 , 0.79375, 0.835  ,
       0.87625, 0.9175 , 0.95875, 1.     ]),
                                        'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'max_features': array([0.1  , 0.325, 0.55 , 0.775, 1.   ]),
                                        'n_estimators': array([ 50,  55,  60,  65,  70,  75,  80,  85,  90,  95, 100, 105, 110,
       115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175,
       180, 185, 190, 195]),
                                        'subsample': array([0.1 , 0.35, 0.6 , 0.85])})

In [5]:
print(model_search.best_params_)

{'subsample': 0.6, 'n_estimators': 155, 'max_features': 0.775, 'max_depth': 8, 'learning_rate': 0.17500000000000002, 'lambda_l2': 0.01}


In [11]:
# Test

metrics = {
    "Name":[],
    "R2":[], 
    "MAE":[], 
    "MSE":[],
    "RMSE":[],
}

In [12]:
preds = model_search.predict(x_test)
metrics['Name'].append("Plain")
metrics['R2'].append(r2_score(preds, y_test))
metrics['MAE'].append(mean_absolute_error(preds, y_test))
metrics['MSE'].append(mean_squared_error(preds, y_test, squared=True))
metrics['RMSE'].append(mean_squared_error(preds, y_test, squared=False))

[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] lambda_l2 is set=0.01, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.01
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [14]:
pd.DataFrame(metrics)

,Name,R2,MAE,MSE,RMSE
0,Plain,0.984504,16.898603,2387.16098,48.858581


### Forecast AC_POWER